In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
kr_pm10_2015 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2015.csv", sep=',',encoding='mbcs')
kr_pm10_2016 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2016.csv", sep=',',encoding='mbcs')
kr_pm10_2017 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2017.csv", sep=',',encoding='mbcs')
kr_pm10_2018 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2018.csv", sep=',',encoding='mbcs')
kr_pm10_2019 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2019.csv", sep=',',encoding='mbcs')
kr_pm10_2020 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2020.csv", sep=',',encoding='mbcs')
kr_pm10_2021 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2021.csv", sep=',',encoding='mbcs')
kr_pm10_2022 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2022.csv", sep=',',encoding='mbcs')
kr_pm10_2023 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2023.csv", sep=',',encoding='mbcs')
kr_pm10_2024 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2024.csv", sep=',',encoding='mbcs')

In [9]:
kr_pm10 = pd.concat([kr_pm10_2015, kr_pm10_2016, kr_pm10_2017, kr_pm10_2018, kr_pm10_2019,
                    kr_pm10_2020, kr_pm10_2021, kr_pm10_2022, kr_pm10_2023, kr_pm10_2024], axis = 0)

In [11]:
kr_pm10 = kr_pm10.drop("지점", axis = 1)

In [13]:
kr_pm10.columns = ["지점명", "일시", "pm10"]

In [25]:
kr_pm10["지점명"].unique()

array(['속초', '광덕산', '대관령', '춘천', '백령도', '서울', '울릉도독도(감)', '관악산', '수원',
       '영월', '울진', '안면도(감)', '추풍령', '안동', '군산', '대구', '전주', '울산', '광주',
       '구덕산', '흑산도', '진도(첨찰산)', '고산', '진주', '강화', '북격렬비도', '천안', '진도군',
       '문경', '연평도', '북춘천'], dtype=object)

## 종속변수 지점 선택

- 추후 어느 지점들을 종속변수로 삼을지 논의가 필요함
    - 가능하다면 초미세먼지 데이터가 있는 곳과 중복되도록 하는게 좋을지도
    - 현재 초미세먼지는 서울만 예측을 하고 여유가 된다면 다른 지역도 시도해보는 방향

- 종속변수 지정을 위해 고려할만한 요소들?
    - 인구가 많은 지점
    - 특정 지역을 대표할 수 있는 지점
    - 관광객 등의 유동인구나 관심이 높은 지점
    
- 종속변수마다 모델을 새로 만들고 학습시켜야 하는 굉장히 복잡한 과정을 거쳐야함
    - 현실적으로 너무 많은 지점을 종속변수로 채택할 수 없음
    - 경기도, 강원도, 충청도, 전라도, 경상도를 대표하는 지점 1개씩 지정하는 방향 고민중

- 각 지점별 메모
    - 수도권 대표로 **서울**
    - **구덕산**은 부산에 있는 산. 경상도 대표로 선정하기 위해서는 구덕산이 적절한 것인지, **울산**으로 대체할 것인지 고민 필요
        - 구덕산의 기상레이더관측소는 고도 391m의 시약산 정상에 위치해있음. 부산의 미세먼지를 반영하기에 적절한가?
        - https://www.newsis.com/view/NISX20180114_0000202464
    - 충청도 대표 **천안**...? 세종이나 대전 있으면 좋을텐데...
    - 전라도 대표 **광주**
    - 강원도 도시 지점인 속초, 춘천, 영월 중에서 인구가 가장 많은 지점은 **춘천**
        - 동해안의 대표로써 **속초**가 지리적으로 적절하다고 생각됨

In [17]:
kr_pm10["일시"] = kr_pm10["일시"].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M'))

In [21]:
kr_pm10_pivot = kr_pm10.pivot(index = ["일시"], columns = ["지점명"], values = ["pm10"])

In [31]:
kr_pm10_pivot = kr_pm10_pivot.loc[:, [("pm10", "서울"), ("pm10", "속초"), ("pm10", "천안"), ("pm10", "광주"), ("pm10", "울산")]]

In [35]:
kr_pm10_pivot.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82143 entries, 2015-01-01 00:00:00 to 2024-05-19 23:00:00
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   (pm10, 서울)  77079 non-null  float64
 1   (pm10, 속초)  77128 non-null  float64
 2   (pm10, 천안)  77396 non-null  float64
 3   (pm10, 광주)  75839 non-null  float64
 4   (pm10, 울산)  78617 non-null  float64
dtypes: float64(5)
memory usage: 3.8 MB
